In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

from grad import norm_of_surface
from vector import Vec
from intercept import within_error, estimate_all_intercepts, nearest_point, is_in_interval

In [2]:
# estimate_all_intercepts(f, g, interval, n_tests = 100, n_iter = 100, error = 1e-10):

N_TESTS = 1000
N_ITER = 100
ERROR = 1e-10

In [3]:
def func1(pos):
    return (pos[0] ** 2 + pos[1] ** 2)


def sign(num):
    val = np.sign(num)
    if val != 0:
        return val
    else:
        return 1

def reflect(inc: Vec, norm: Vec) -> Vec:
    
    inc.normalise()
    norm.normalise()
    
    A = inc.dot(norm)
    return inc.add(-norm.times(2 * A)).normalise()

def refract(inc: Vec, norm: Vec, n1, n2):
    
    inc.normalise()
    norm.normalise()
    
    A = inc.dot(norm)
    A2 = A * A
    mu = n1 / n2
    mu2 = mu * mu
    
    
    cp1 = norm.times( sign(A) * (1 - mu2 * (1 - A2)) ** 0.5 - (A * mu))
    cp2 = inc.times(mu)
    
    return cp1.add(cp2).normalise()



In [ ]:
class Ray:
    def __init__(self, direction: Vec, start_pos: Vec, start_boundary = None, level = 0):
        
        self.direction = direction
        self.direction.normalise()
        
        self.start_pos = start_pos
        
        # If start_boundary = None, then treat the ray to be in air (n = 1)
        self.start_boundary = start_boundary
        
        # The number of interactions have happened before this ray's existance
        # Very first ray is level = 0, reflected or refracted is level = 1, next interaction is level = 2
        self.level = level
        
        self.end_boundary = None
        self.end_pos = None
        self.refracted_ray = None
        self.reflected_ray = None
        
    # Finds distance from start position given 1 coordinate from one of x, y or z
    def where(self, x = None, y = None, z = None):
        
        where_pos = np.array([x, y, z])
        val_arg = 0
        
        None_count = 0
        for i, val in enumerate(where_pos):
            if val is None:
                None_count++
            else:
                val_arg = i
                
        if None_count != 2:
            return None
            
            
        # Only 1 value to test
        
        
        if self.direction.vec[val_arg] == 0:
            return None
        
        dist = (where_pos[val_arg] - self.start_pos.vec[val_arg]) / self.direction.vec[val_arg]
        
        # Cant have points before the starting position
        if dist < 0:
            return None
        
        return dist
        
        
    def pos_from_distance(self, dist):
        return self.start_pos.add( self.direction.times(dist) )
        
        
        
        
        
    
    def update_ray(self, clipping_distance = 1e-9):
        
        # Must have an end boundary (need to know norm and refractive index)
        if self.end_boundary is None:
            return False
        
        self.end_pos = self.end_boundary.find_intercept(self)
        
        # Must have an end position (None suggests it goes to inf)
        if self.end_pos is None:
            return False
        
        
        
        # Add a very small distance to stop ray clipping to boundaries
        refracted_pos = self.end_pos + clipping_distance * self.direction
        reflected_pos = self.end_pos - clipping_distance * self.direction
        
        # Initial refractive index
        n1 = 1
        if self.start_boundary is None:
            n1 = 1
        else:
            n1 = self.start_boundary.get_n()
        
        # The refractive index of the new medium 
        n2 = self.end_boundary.get_n()
        
        
        self.reflected_ray = Ray( 
            reflect(
                self.direction, 
                self.end_boundary.get_norm(self.end_pos)
            ),
            Vec(reflected_pos),
            # The ray is in the same medium
            start_boundary = self.start_boundary,
            self.level+1
        )
        
        self.refracted_ray = Ray( 
            refract(
                self.direction, 
                self.end_boundary.get_norm(self.end_pos),
                n1,
                n2
            ),
            Vec(refracted_pos),
            # The ray has entered the new medium
            start_boundary = self.end_boundary,
            self.level+1
        )
        
        return True 


class Bbox:
    def __init__(self, coords):
        
        self.bbox = coords
        
        if len(self.bbox) != 2:
            exit()
        else:
            for coord in self.bbox:
                if len(coord) != 2:
                    exit()
        
    def inbound_range(self, ray: Ray):
        
        ray_start_dist = 0
        
        left_intercept_dist = ray.where(x = self.bbox[0][0])
        right_intercept_dist = ray.where(x  = self.bbox[0][1])
        
        top_intercept_dist = ray.where(y = self.bbox[1][0])
        bottom_intercept_dist = ray.where(y = self.bbox[1][1])
        
        points_inside = [self.xy_val_in(self, ray.pos_from_distance(ray_start_dist).vec),
                         self.x_val_in(self, ray.pos_from_distance(left_intercept_dist).x())
                         self.x_val_in(self, ray.pos_from_distance(right_intercept_dist).x())
                         self.y_val_in(self, ray.pos_from_distance(top_intercept_dist).y())
                         self.y_val_in(self, ray.pos_from_distance(bottom_intercept_dist).y())
            
        ]
        
    def x_val_in(self, val):
        return(val >= self.bbox[0][0] and val <= self.bbox[0][1])
    
    def y_val_in(self, val):
        return(val >= self.bbox[1][0] and val <= self.bbox[1][1])
    
    def xy_val_in(self, vals):
        return(self.x_val_in(self, vals[0]) and self.y_val_in(self, vals[1]))
        
        

class Rule:
    def __init__(self, func, bbox):
        self.func = func
        self.bbox = bbox
        
        
        

# Class to define the ranges/intervals a boundary/plane is bound by
class Interval:
    def __init__(self, *rules):
        
        self.rules = rules
        
        self.bbox = np.array([None, None], [None, None])
        
        for i, rule in enumerate(self.rules):
            if i == 0:
                self.bbox = rule.bbox
            else:
                # Take minimum x
                self.bbox[0][0] = min(self.bbox[0][0], rule.bbox[0][0])
                # Take minimum y
                self.bbox[0][1] = min(self.bbox[0][1], rule.bbox[0][1])
                # Take maximum x
                self.bbox[1][0] = max(self.bbox[1][0], rule.bbox[1][0])
                # Take maximum y
                self.bbox[1][1] = max(self.bbox[1][1], rule.bbox[1][1])
            
    
    # Must have all rules satisfied
    def isin_interval(self, pos):
        
        for rule in self.rules:
            if rule.func(pos) == False:
                return False
        return True

class Func_Area:
    def __init__(self, 
                 interval, # The interval MUST only bound the domain the function exists, and NOT bound the result.
                 func, 
                 is_less_than = True):
        
        # The rules that define if a point is included on the plane defined by func
        self.interval = interval
        
        # Function of surface bound by an interval
        self.func = func
        
        # Is the included volume above or below the function
        self.is_less_than = is_less_than
        
        # The volume which owns this area
        self.parent = None
        
        
        
        
    def set_parent(self, parent):
        self.parent = parent
    
    def isin_domain(self, pos):
        return self.interval.isin_interval(pos)
    
    def isin_volume(self, pos):
        
        if not self.isin_domain(pos):
            return False
        
        if (pos[2] < self.func(pos)) == self.is_less_than:
            return True
        
        return False
    
    
    def find_intercepts(self, ray: Ray):
        




class Volume:
    def __init__(self, n = 1, *boundaries):
        self.n = n
        self.bounds = boundaries
        
        for bound in self.bounds:
            bound.set_parent(self)
        
    
    def find_intercept(self, ray):
        

class Scene:
    def __init__(self, *areas):
        self.areas = areas
        
        
    def trace_ray(self, ray):

In [ ]:
def Circule_Rule(pos):
    
    if (pos[0] ** 2 + pos[1] ** 2) <= 1:
        return True
    
    return False

In [4]:
a = norm_of_surface(func1, [0, 0])

In [5]:
a.vec

array([-1.e-10, -1.e-10,  1.e+00])